# AI EU Project #

In [4]:
import numpy as np
import pandas as pd

# Load the dataset
df = pd.read_excel('pstw_dataset.xlsx')

## Drop unnused columns ##

In [6]:
# Ensures all the projects are within AI 
df = df[df['Primary Technology'].str.lower() == 'artificial intelligence']

# All columns needed for the project
columns = [
    "Geographical extent",
    "Geographical coverage (country)",
    #"Responsible organisation",
    "Responsible organisation category",
    "Functions of Government (COFOG level I)",
    "Functions of Government (COFOG level II)",
    " Status",
    "Process type",
    "Application type",
    "Cross Border",
    "Cross Sector",
    "Interaction",
    "Improved Public Service",
    "Personalized Services",
    "Public (citizen)-centered services",
    "Increase quality of PSI and services",
    "More responsive, efficient, and cost-effective public services",
    "New services or channels",
    "Improved Administrative Efficiency",
    "Cost-reduction",
    "Responsiveness of government operation",
    "Improved management of public resources",
    "Increased quality of processes and systems",
    "Open government capabilities",
    "Increased transparency of public sector operations",
    "AI Classification (I)",
    "AI Classification Subdomain (II) (main)",
    "AI Classification Subdomain (II) (Other I)",
    "AI Keywords",
    "Collaboration type",
    "Funding source"
]

# Columns to replace that contains X:s
x_columns = [
    "Improved Public Service",
    "Personalized Services",
    "Public (citizen)-centered services",
    "Increase quality of PSI and services",
    "More responsive, efficient, and cost-effective public services",
    "New services or channels",
    "Improved Administrative Efficiency",
    "Cost-reduction",
    "Responsiveness of government operation",
    "Improved management of public resources",
    "Increased quality of processes and systems",
    "Open government capabilities",
    "Increased transparency of public sector operations"
]

# Selects the right columns
df_selected = df[columns]

# Replaces X/ x with 1 or 0 to ease dummy-encoding
def translate_x(df, column):
  df[column] = df[column].replace({
      "x": 1,
      "X": 1,
      "\xa0": 0,
      np.nan: 0
  })
  df[column] = df[column].astype(int)
  #return df[column]


            
# Replaces Pilot phase with pilot for consistensy
df[' Status'] = df[' Status'].replace({'Pilot phase': 'Pilot'})

for column in x_columns:
    print(column)
    translate_x(df_selected, column)

# Dummy-encodes all other columns
for col in columns:
    if col != " Status" and col not in x_columns:
        dummies = pd.get_dummies(df, columns=[col], prefix="de-", drop_first=True)
        df = pd.concat([df, dummies], axis=1)
    df.drop(col, axis=1, inplace=True)
    
print(df_selected)

AttributeError: 'DataFrame' object has no attribute 'str'